In [1]:
path = "/ssd/zhuang/dataset/data_KeSpeech/KeSpeech/Audio/1005697/phase1/1005697_032baa35.wav"

In [4]:
import os

import fairseq
import numpy as np
import torch
import tqdm
import yaml
import json
from npy_append_array import NpyAppendArray

from mfcc_extractor import load_audio

# 读取配置文件
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)


class Reader(object):
    def __init__(self, ckpt_path, layer, max_chunk=1600000):
        model, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([ckpt_path])
        self.model = model[0].eval().cuda()
        self.task = task
        self.layer = layer
        self.max_chunk = max_chunk

    def get_feats(self, path):
        waveform, _ = load_audio(path)
        feat = []
        feat_chunk, _ = self.model.extract_features(source=waveform.cuda(),
                                                    padding_mask=None,
                                                    mask=False,
                                                    output_layer=self.layer)
        feat.append(feat_chunk)
        return torch.cat(feat, 1).squeeze(0)




ckpt_path = config['KMEANS']['ckpt_path']  # hubert的模型
layer = config['KMEANS']['layer']  # 需要从第几层提取
max_chunk = config['KMEANS']['max_chunk']  # 每次提取的最大长度

# # 初始化reader
# reader = Reader(ckpt_path, layer, max_chunk)
# feat = reader.get_feats(path)


In [5]:
ckpt_path

'/data/NAS_PLUS/zhuang/CodeBase/008_simple_kmeans/save/hubert_base_ls960.pt'

In [6]:
layer

9

In [7]:
max_chunk

1600000

In [8]:
reader = Reader(ckpt_path, layer, max_chunk)

/tmp/ipykernel_2831143/729559370.py:1: UserWarning: 
'config' is validated against ConfigStore schema with the same name.
This behavior is deprecated in Hydra 1.1 and will be removed in Hydra 1.2.
See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/automatic_schema_matching for migration instructions.
  reader = Reader(ckpt_path, layer, max_chunk)
2024-08-01 15:53:30 | INFO | fairseq.tasks.hubert_pretraining | current directory is /ssd/zhuang/code/FunASR/examples/data_toolkit/kmeans_label
2024-08-01 15:53:30 | INFO | fairseq.tasks.hubert_pretraining | HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': '/checkpoint/wnhsu/data/librispeech/960h/iter/250K_50hz_km100_mp0_65_v2', 'fine_tuning': False, 'labels': ['layer6.km500'], 'label_dir': None, 'label_rate': 50.0, 'sample_rate': 16000, 'normalize': False, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 250000, 'min_sample_size': 32000, 'single_target': False, 'random_crop': True, 'pad_audio': False}
2024

In [10]:
reader.get_feats(path).size()

torch.Size([220, 768])

In [11]:
import torchaudio
audio  , _ = torchaudio.load(path)
print(audio.size())

torch.Size([1, 70698])


In [14]:
path= "/ssd/zhuang/dataset/data_KeSpeech/KeSpeech/Audio/1006957/phase1/1006957_a1f1dcc7.wav"

In [15]:
reader.get_feats(path).size()

torch.Size([220, 768])

In [16]:
import torchaudio
audio  , _ = torchaudio.load(path)
print(audio.size())

torch.Size([1, 70698])


In [26]:
import os
root ="/ssd/zhuang/code/FunASR/examples/kespeech/DATA/data3"

In [27]:
for root, dirs, files in os.walk(root):
    for file in files:
        if "hubert" in file:
            print (os.path.join(root, file) ,"is removing")
            os.remove(os.path.join(root, file))
        if "kmeans" in file:
            print (os.path.join(root, file), "is removing")
            os.remove(os.path.join(root, file))


/ssd/zhuang/code/FunASR/examples/kespeech/DATA/data3/AS/test/kmeans_label is removing
/ssd/zhuang/code/FunASR/examples/kespeech/DATA/data3/AS/test/hubert_9layer.npy is removing
/ssd/zhuang/code/FunASR/examples/kespeech/DATA/data3/AS/test/hubert_9layer.len is removing
/ssd/zhuang/code/FunASR/examples/kespeech/DATA/data3/AS/dev/kmeans_label is removing
/ssd/zhuang/code/FunASR/examples/kespeech/DATA/data3/AS/dev/hubert_9layer.npy is removing
/ssd/zhuang/code/FunASR/examples/kespeech/DATA/data3/AS/dev/hubert_6layer.npy is removing
/ssd/zhuang/code/FunASR/examples/kespeech/DATA/data3/AS/dev/hubert_9layer.len is removing
/ssd/zhuang/code/FunASR/examples/kespeech/DATA/data3/AS/dev/hubert_6layer.len is removing
/ssd/zhuang/code/FunASR/examples/kespeech/DATA/data3/AS/train/kmeans_label is removing
/ssd/zhuang/code/FunASR/examples/kespeech/DATA/data3/AS/train/hubert_9layer.npy is removing
/ssd/zhuang/code/FunASR/examples/kespeech/DATA/data3/AS/train/hubert_9layer.len is removing
/ssd/zhuang/code